In [3]:
import akshare as ak

# fund_name_em_df = ak.fund_name_em()
f = ak.fund_individual_basic_info_xq(symbol="004814")
ak.fund_portfolio_hold_em(symbol="004814", date=2025)

,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,1,01336,新华保险,6.39,1690.43,46250.16,2025年1季度股票投资明细
1,2,601899,紫金矿业,6.13,2448.08,44359.25,2025年1季度股票投资明细
2,3,00939,建设银行,5.00,5691.60,36141.66,2025年1季度股票投资明细
3,4,02628,中国人寿,4.84,2532.00,35042.88,2025年1季度股票投资明细
4,5,000951,中国重汽,4.72,1691.36,34131.56,2025年1季度股票投资明细
...,...,...,...,...,...,...,...
125,126,000951,中国重汽,2.51,2810.88,47503.94,2025年4季度股票投资明细
126,127,600031,三一重工,2.31,2067.79,43692.41,2025年4季度股票投资明细
127,128,00883,中国海洋石油,2.23,2190.20,42139.45,2025年4季度股票投资明细
128,129,002142,宁波银行,2.22,1493.94,41964.85,2025年4季度股票投资明细


In [8]:
import datetime
import pandas as pd
import akshare as ak


def _resolve_fund(query: str) -> dict:
    """根据名称或代码解析基金"""
    fund_list = ak.fund_name_em()
    query = str(query).strip()

    if query.isdigit():
        match = fund_list[fund_list["基金代码"] == query]
    else:
        match = fund_list[fund_list["基金简称"].str.contains(query, na=False)]

    if match.empty:
        raise ValueError(f"未找到匹配基金: {query}")

    row = match.iloc[0]
    return {
        "code": row["基金代码"],
        "name": row["基金简称"],
        "type": row["基金类型"],
    }


def _latest_nav_open_fund(code: str) -> dict:
    """开放式基金最新净值"""
    nav_df = ak.fund_open_fund_info_em(symbol=code, indicator="单位净值走势")
    latest = nav_df.iloc[-1]
    return {"date": latest["净值日期"], "nav": latest["单位净值"]}


def _latest_nav_money_fund(code: str) -> dict:
    """货币型基金最新净值"""
    nav_df = ak.fund_money_fund_info_em(symbol=code)
    latest = nav_df.iloc[-1]
    return {"date": latest["净值日期"], "nav": latest["每万份收益"], "nav_7d": latest["7日年化收益率"]}


def _extract_quarter(value: str, year: int) -> int | None:
    """从季度描述中提取季度数"""
    if not isinstance(value, str):
        return None
    if f"{year}年" not in value:
        return None
    for q in range(4, 0, -1):
        if f"{q}季度" in value:
            return q
    return None


def _latest_quarter_holdings_by_year(code: str, year: int) -> pd.DataFrame | None:
    """获取指定年份最后一个季度的持仓数据"""
    holdings = ak.fund_portfolio_hold_em(symbol=code, date=year)
    if holdings.empty:
        return None

    quarters = holdings["季度"].apply(lambda x: _extract_quarter(x, year))
    if quarters.isna().all():
        return None

    latest_quarter = int(quarters.dropna().max())
    return holdings[holdings["季度"].str.contains(f"{year}年{latest_quarter}季度", na=False)]


def _latest_quarter_holdings(code: str) -> pd.DataFrame | None:
    """当前年份无数据时回退上一年"""
    year = datetime.datetime.now().year
    for y in [year, year - 1]:
        result = _latest_quarter_holdings_by_year(code, y)
        if result is not None and not result.empty:
            return result
    return None


def _format_holdings(holdings: pd.DataFrame | None) -> pd.DataFrame | None:
    """以可读方式输出持仓与占比"""
    if holdings is None or holdings.empty:
        return None

    output = holdings[["季度", "股票代码", "股票名称", "占净值比例", "持仓市值"]].copy()
    output["占净值比例"] = output["占净值比例"].astype(float).map(lambda x: f"{x:.2f}%")
    output["持仓市值"] = output["持仓市值"].astype(float).map(lambda x: f"{x:,.2f}")
    return output


def get_fund_snapshot(query: str) -> dict:
    """输入名称或代码，自动判断类型并获取最新净值与持仓"""
    meta = _resolve_fund(query)

    if "货币型" in meta["type"]:
        nav = _latest_nav_money_fund(meta["code"])
    else:
        nav = _latest_nav_open_fund(meta["code"])

    holdings = _latest_quarter_holdings(meta["code"])

    return {
        "code": meta["code"],
        "name": meta["name"],
        "type": meta["type"],
        "nav": nav,
        "holdings": holdings,
        "holdings_readable": _format_holdings(holdings),
    }


# 示例：输入名称或代码
result = get_fund_snapshot("004814")
result["nav"], result["holdings_readable"].head() if result["holdings_readable"] is not None else None


({'date': datetime.date(2026, 1, 30), 'nav': np.float64(2.3934)},
                  季度    股票代码  股票名称  占净值比例        持仓市值
 119  2025年4季度股票投资明细   02318  中国平安  5.81%  109,913.12
 120  2025年4季度股票投资明细   02628  中国人寿  5.67%  107,211.97
 121  2025年4季度股票投资明细   01336  新华保险  5.39%  102,077.26
 122  2025年4季度股票投资明细  601899  紫金矿业  4.90%   92,812.44
 123  2025年4季度股票投资明细  000425  徐工机械  3.05%   57,654.40)

In [1]:
import akshare as ak

ak.stock_bid_ask_em(symbol="300502")
# ak.stock_hk_hist_min_em(symbol="01611")

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))